In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -U sentence-transformers
!pip install evaluate
!pip install transformers[torch]

In [ ]:
# dataset
!gdown 1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK

!gdown 1vzYpVcquBvzX5Ige3klpaACQFbjEP4Ak

!gdown 1yBppNyzNCS5tinBvlTIyuMbBDmQhmKBF
!gdown 1GvsfK3vZIBbYViI-KFPCsW-mFw4RUjqK

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import evaluate

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

import re

import json

In [ ]:
data = pd.read_csv('data_corrected_spell.csv')

In [ ]:
def remove_extra_symbols(text):
    # Убираем лишние символы в начале предложения, если модель их добавила
    # correct_text = correct_text[correct_text.index(input_text[0]):]
    text = text.lstrip('.,[]«»')

    # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
    text = re.sub(r'([^\w\s])\1+', r'\1', text)

    return text

data['Текст инцидента'] = data['Текст инцидента'].apply(remove_extra_symbols)

In [ ]:
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу


In [ ]:
def get_id_and_labels():
    id2label_path = 'id2label.json'
    label2id_path = 'label2id.json'

    with open(id2label_path, 'r', encoding='UTF-8') as file:
        id2label = json.load(file)

    id2label = {int(key):value for key,value in id2label.items()}

    with open(label2id_path, 'r', encoding='UTF-8') as file:
        label2id = json.load(file)

    return id2label, label2id

id2label, label2id = get_id_and_labels()

In [ ]:
data['label'] = [label2id[topic] for topic in data['Тема']]
data

,Исполнитель,Группа тем,Текст инцидента,Тема,label
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,1
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,2
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,3
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,4
...,...,...,...,...,...
22485,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,125
22486,Губахинский городской округ,ЖКХ,Город Гремячинск — ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,44
22487,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...",Технические проблемы с записью на прием к врачу,4
22488,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов?! Воо...,Благоустройство придомовых территорий,122


In [ ]:
# checkpoint = "xlm-roberta-base"
# checkpoint = "cointegrated/rubert-tiny2"
checkpoint = "ai-forever/ruBert-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels = data['Тема'].unique().shape[0],
    id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data, val_data = train_test_split(
    data[['Текст инцидента', 'label']], random_state=42, test_size=.1
)

In [ ]:
train_data

,Текст инцидента,label
13061,"Эти животные ни когда трубку не берите, либо с...",4
1166,Какое количество привитых граждан заболели пов...,47
20482,"Добрый день. Скажите, какие документы нужны дл...",2
5068,"Когда шла стройка ГРЕС, население в Яйве =1500...",40
7685,Прошу повлиять на УК «Губерния». Лёд и снег уп...,14
...,...,...
11964,Добрый день. С кем можно посоветоваться по пов...,57
21575,Добрый день! Подавала на выплату с 3 до 7 лет....,2
5390,"Это цифры не по Пермскому краю! Непонятно, поч...",52
860,"Глава района новый, видимо, ещё не в курсе, чт...",9


In [ ]:
class TextDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences = data_df["Текст инцидента"].values
        self.labels = data_df['label'].values

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, i):
        sentence, label = self.sentences[i], self.labels[i]

        tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens['labels'] = label

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        # tokens['labels'] = tokens['labels'].to(torch.float)

        return tokens


train_dataset = TextDataset(train_data, tokenizer)
val_dataset = TextDataset(val_data, tokenizer)

# train_dataset[0]

In [ ]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    out = {}

    out.update(accuracy.compute(predictions=predictions, references=labels))
    out.update(f1_metric.compute(predictions=predictions, references=labels,
                                 average='weighted'))

    return out

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(None, classes=np.array(list(label2id.keys())), y=data["Тема"])

class_weights = torch.tensor(class_weights, device=model.device).to(torch.float).to("cuda")

In [ ]:
from torch import nn


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="promobot/models/ruBert-base",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.238800,2.142157,0.498888,0.429913
2,1.723300,1.960025,0.534460,0.475102


TrainOutput(global_step=10122, training_loss=2.270326754199354, metrics={'train_runtime': 4646.6196, 'train_samples_per_second': 8.712, 'train_steps_per_second': 2.178, 'total_flos': 1.0669718985689088e+16, 'train_loss': 2.270326754199354, 'epoch': 2.0})

In [ ]:
class_weights = compute_class_weight("balanced", classes=np.array(list(label2id.keys())), y=data["Тема"])

class_weights = torch.tensor(class_weights, device=model.device).to(torch.float).to("cuda")

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="promobot/models/ruBert-base",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.039700,2.565021,0.547799,0.520341


TrainOutput(global_step=5061, training_loss=2.20413623772881, metrics={'train_runtime': 2327.8108, 'train_samples_per_second': 8.695, 'train_steps_per_second': 2.174, 'total_flos': 5334859492844544.0, 'train_loss': 2.20413623772881, 'epoch': 1.0})

In [ ]:
model.save_pretrained("ruBert-base/")

In [ ]:
!zip -r ruBert-base.zip ruBert-base

  adding: ruBert-base/ (stored 0%)
  adding: ruBert-base/config.json (deflated 88%)
  adding: ruBert-base/model.safetensors (deflated 7%)


In [ ]:
!cp ruBert-base.zip /content/drive/MyDrive/

### try running it

In [ ]:
!gdown 1Uj_qnIfoxgMUuWF6_hwFN3yb3ILNXS8H

Downloading...
From: https://drive.google.com/uc?id=1Uj_qnIfoxgMUuWF6_hwFN3yb3ILNXS8H
To: /content/topic2big_topic.json
100% 21.4k/21.4k [00:00<00:00, 45.3MB/s]


In [ ]:
with open("topic2big_topic.json", "r", encoding="UTF8") as f:
    topic2big_topic = json.load(f)

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification",
                      model="ruBert-base",
                      tokenizer="ai-forever/ruBert-base")

In [ ]:
n = np.random.randint(data.shape[0])

print(n)

print(data["Текст инцидента"][n], )
print(data["Тема"][n], '|', data["Группа тем"][n])

label = classifier(data["Текст инцидента"][n])[0]["label"]

print(label, '|', topic2big_topic[label])

11394
В Голованово не только в частном секторе, но и среди 2-3 этажных домов тоже нет освещения. Сегодня бабушка шла вечером домой и в потемках упала в лужу. Темно очень.
Отсутствие фонарей освещения | Благоустройство
Освещение неисправно или отсутствует | Дороги


### test

In [ ]:
val_data

,Текст инцидента,label
4003,"'Здравствуйте,на против церкви на Чапаева не р...",127
3615,"'Здравствуйте!<br>Ввиду того, что на мосту Кам...",157
7907,'Уважаемая Администрация! Что опять на автобус...,87
1398,"'Здравствуйте, скажите пожалуйста более точнее...",1
12113,"'Здравствуйте,хотелось бы получить ответ на во...",22
...,...,...
13852,'А на детей с 8 лет каковы выплаты?,2
274,"'Здравствуйте ,как быть подскажите ,у меня 1/4...",5
14145,'Здравствуйте!,1
8465,"'Добрый день! Скажите пожалуйста, к кому обращ...",125


In [ ]:
n = np.random.randint(val_data.shape[0])
n

409

In [ ]:
sentence = str(val_data["Текст инцидента"].iloc[n])

print(sentence, '\n', id2label[val_data["label"].iloc[n]])

tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=512)

tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}


for key in tokens:
    tokens[key] = tokens[key].to("cuda").unsqueeze(0)

pred = model(**tokens)

'Добрый день. Проживаю в городе Лысьва в своём доме. За забором на перекрестке стоит колодец водоканала. Весной всегда он переполняется и вода под землёй поступает в наши колодцы. Каждый год звоню в водоканал, всегда приезжали и выкачивали. Сегодня сообщили что этот калодец частный, т.е. мой, на балансе водоканала не числится. Как так? Был их, стал мой. Разъясняющих ответов от руководства водоканала не услышала, просто бросили трубку. Хочу справедливости. Получается целая улица просто так пьёт воду больше 20 лет? 
 Подключение к водоснабжению


In [ ]:
id2label[pred["logits"].argmax().item()]

'★ Открытые канализационные люки'